In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [1]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import json

In [2]:
#Tentando pegar os links da página, mas ainda estou com problemas no xpath
#http://www4.planalto.gov.br/legislacao/portal-legis/legislacao-1/leis-ordinarias
url = 'http://www4.planalto.gov.br/legislacao/portal-legis/legislacao-1/leis-ordinarias/2020-leis-ordinarias'
#instanciando o firefox
option = Options()
option.headless = True
driver = webdriver.Firefox(options=option)
driver.get(url)
path = "//div[@id='visao2']//table//tbody//tr//td//a[@href]"
#//a[@href]
#driver.find_element_by_xpath(path[1]).click()
#--aqui--#
#time.sleep(10)
links = driver.find_elements_by_xpath(path)
lista = []
for a in driver.find_elements_by_xpath(path):
    print(a.get_attribute('href'))
    lista.append(a)
#links_hrefs = [link.get_attribute('href') for link in links]
#lista = []
#lista.append(links_hrefs)

In [14]:
url = 'http://www4.planalto.gov.br/legislacao/portal-legis/legislacao-1/leis-ordinarias/2020-leis-ordinarias'
url2 = 'http://www.planalto.gov.br/ccivil_03/_ato2019-2022/2020/lei/L14013.htm'
option = Options()
option.headless = True
driver = webdriver.Firefox(options=option)
driver.get(url2)
#--pegando o código fonte inteiro da página --#
elements = driver.page_source
print(elements)
driver.quit()

<html xmlns="http://www.w3.org/TR/REC-html40" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:v="urn:schemas-microsoft-com:vml"><head>
  <meta content="Microsoft FrontPage 6.0" name="GENERATOR">
  <title>
   L14013
  </title>
  <style>
   <!--
 table.MsoNormalTable
	{mso-style-parent:"";
	font-size:10.0pt;
	font-family:"Times New Roman"
	}
h1
	{margin-bottom:.0001pt;
	text-align:justify;
	line-height:12.0pt;
	page-break-after:avoid;
	font-size:12.0pt;
	font-family:"Times New Roman";
	font-weight:normal; margin-left:0pt; margin-right:0pt; margin-top:0pt}
h2
	{margin-top:12.0pt;
	margin-right:0pt;
	margin-bottom:3.0pt;
	margin-left:0pt;
	page-break-after:avoid;
	font-size:12.0pt;
	font-family:"Times New Roman";
	font-weight:normal}
span.Hiperlink
	{mso-style-parent:"";
	color:blue;
	text-decoration:underline;
	text-underline:single}
h6
	{margin-bottom:.0001pt;
	text-align:center;
	line-height:150%;
	page-break-after:avoid;
	font-size:12.0pt;
	font-family:Arial;
	margin-left:0cm; 

In [15]:
#2. Parsear o conteúdo
soup = BeautifulSoup(elements, 'lxml')

In [16]:
#myp = soup.findAll('p', class_='Artigo') neste caso a class é outra
myp = soup.findAll('p', class_='texto1')
myp

[<p align="justify" class="texto1" style="text-indent: 1cm; margin-top: 20px; margin-bottom: 20px">
 <span style="font-size: 10.0pt; font-family: Arial,sans-serif; color: black">
 <a name="art1"></a>Art. 1</span><span style='font-size:10.0pt;
 font-family:"Arial","sans-serif";color:black'>º  No mês de 
 	janeiro de 2020, o salário-mínimo será de R$ 1.039,00 (mil e trinta e nove 
 	reais).</span></p>,
 <p align="justify" class="texto1" style="text-indent: 1cm; margin-top: 20px; margin-bottom: 20px">
 <span style='font-size:10.0pt;font-family:"Arial","sans-serif";color:black'>
 	Parágrafo único. Em decorrência do disposto no <b>caput</b> deste artigo, o 
 	valor diário do salário-mínimo corresponderá, no mês de janeiro de 2020, a 
 	R$ 34,63 (trinta e quatro reais e sessenta e três centavos) e o valor 
 	horário, a R$ 4,72 (quatro reais e setenta e dois centavos).</span></p>,
 <p align="justify" class="texto1" style="text-indent: 1cm; margin-top: 20px; margin-bottom: 20px">
 <span style=

In [33]:
lista = []
for parag in soup.find_all('p', class_='texto1'):
    for sp in parag.find_all('span'):
        print(sp.text)
        lista.append(sp.text)


Art. 1
º  No mês de 
	janeiro de 2020, o salário-mínimo será de R$ 1.039,00 (mil e trinta e nove 
	reais).

	Parágrafo único. Em decorrência do disposto no caput deste artigo, o 
	valor diário do salário-mínimo corresponderá, no mês de janeiro de 2020, a 
	R$ 34,63 (trinta e quatro reais e sessenta e três centavos) e o valor 
	horário, a R$ 4,72 (quatro reais e setenta e dois centavos).



	Art. 2º A partir de 1º de fevereiro de 2020, o salário-mínimo será de R$ 
	1.045,00 (mil e quarenta e cinco reais).

	Parágrafo único. Em decorrência do disposto no caput deste artigo, o 
	valor diário do salário-mínimo corresponderá, a partir de 1º de fevereiro de 
	2020, a R$ 34,83 (trinta e quatro reais e oitenta e três centavos) e o valor 
	horário, a R$ 4,75 (quatro reais e setenta e cinco centavos).



	Art. 3º Fica revogada a Medida Provisória nº 916, de 31 de dezembro de 2019.



	Art. 4º Esta Lei entra em vigor na data de sua publicação. 


In [35]:
for i in range(len(lista)):
    lista[i] = lista[i].replace('\n\t', '')
    lista[i] = lista[i].replace('\n', '')
    lista[i] = lista[i].replace('\xa0', '')
lista[9]

'Art. 4º Esta Lei entra em vigor na data de sua publicação.'

In [36]:
lista

['Art. 1',
 'º No mês de janeiro de 2020, o salário-mínimo será de R$ 1.039,00 (mil e trinta e nove reais).',
 'Parágrafo único. Em decorrência do disposto no caput deste artigo, o valor diário do salário-mínimo corresponderá, no mês de janeiro de 2020, a R$ 34,63 (trinta e quatro reais e sessenta e três centavos) e o valor horário, a R$ 4,72 (quatro reais e setenta e dois centavos).',
 '',
 'Art. 2º A partir de 1º de fevereiro de 2020, o salário-mínimo será de R$ 1.045,00 (mil e quarenta e cinco reais).',
 'Parágrafo único. Em decorrência do disposto no caput deste artigo, o valor diário do salário-mínimo corresponderá, a partir de 1º de fevereiro de 2020, a R$ 34,83 (trinta e quatro reais e oitenta e três centavos) e o valor horário, a R$ 4,75 (quatro reais e setenta e cinco centavos).',
 '',
 'Art. 3º Fica revogada a Medida Provisória nº 916, de 31 de dezembro de 2019.',
 '',
 'Art. 4º Esta Lei entra em vigor na data de sua publicação.']

In [80]:
lista_artigos = []
lista_colunas = []
for artigo in lista:
    if 'Art.' in artigo:
        lista_colunas.append(artigo[0:7])
        lista_artigos.append(artigo[7:])
        print(artigo)

Art. 1
Art. 2º A partir de 1º de fevereiro de 2020, o salário-mínimo será de R$ 1.045,00 (mil e quarenta e cinco reais).
Art. 3º Fica revogada a Medida Provisória nº 916, de 31 de dezembro de 2019.
Art. 4º Esta Lei entra em vigor na data de sua publicação.


In [79]:
#dicionario = {lista_colunas : lista_artigos}
#df = pd.DataFrame(lista_artigos)
lista_artigos

['',
 ' A partir de 1º de fevereiro de 2020, o salário-mínimo será de R$ 1.045,00 (mil e quarenta e cinco reais).',
 ' Fica revogada a Medida Provisória nº 916, de 31 de dezembro de 2019.',
 ' Esta Lei entra em vigor na data de sua publicação.']

In [62]:
df = pd.DataFrame(lista_artigos).T
df.columns = lista_colunas

In [77]:
df

,Art. 1,Art. 2º,Art. 3º,Art. 4º
0,,"A partir de 1º de fevereiro de 2020, o salári...","Fica revogada a Medida Provisória nº 916, de ...",Esta Lei entra em vigor na data de sua public...
